<a href="https://colab.research.google.com/github/oaldaoshy/Text-Summerizer/blob/main/text_suumarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
from rouge import FilesRouge



In [ ]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences


In [ ]:

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 


In [ ]:
def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix



In [ ]:
def generate_summary(file_name, top_n=7):
    stop_words = stopwords.words('english')
    summarized_text = []

    print("original Text:\n")
    sentences =  read_article("/content/fb.txt")

    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("\nIndexes of top ranked_sentence order:\n \n ", ranked_sentence)    

    for i in range(top_n):
      summarized_text.append(" ".join(ranked_sentence[i][1]))

    summary = ". ".join(summarized_text)
    return str(summary)

In [ ]:
with open('fb_pred.txt', 'w') as f:
    s = str(generate_summary(" ",5))
    f.write(s)
    f.close()

In [ ]:
file1 = input("Enter name/path of reference summary file: ")
file2 = input("Enter name/path of predection summary file: ")

f1 = open(file1, 'r')
f2 = open(file2, 'r')

f1lines = f1.readlines()
f1.close()
f2lines = f2.readlines()
f2.close()

from rouge import Rouge 

rouge = Rouge()
scores = rouge.get_scores(f1lines, f2lines)
print(scores)

Enter name/path of reference summary file: /content/fb_ref.txt
Enter name/path of predection summary file: /content/fb_pred.txt
[{'rouge-1': {'r': 0.7011494252873564, 'p': 0.6559139784946236, 'f': 0.6777777727833334}, 'rouge-2': {'r': 0.5904761904761905, 'p': 0.5486725663716814, 'f': 0.5688073344562747}, 'rouge-l': {'r': 0.7011494252873564, 'p': 0.6559139784946236, 'f': 0.6777777727833334}}]
